In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.auto import tqdm
from glob import glob
import time, gc
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# from tensorflow import keras
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator

from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib import pyplot as plt
import tensorflow as tf
import seaborn as sns


from keras.layers import Input
from keras.layers import Conv2D, MaxPool2D, Dense, BatchNormalization, Activation, add, GlobalAvgPool2D, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras import regularizers
from keras.utils import plot_model
from keras import backend as K
from keras import models


import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train_df_ = pd.read_csv('/kaggle/input/bengaliai-cv19/train.csv')
train_df_ = train_df_.drop(['grapheme'], axis=1, inplace=False)
train_df_[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']] = train_df_[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']].astype('uint8')
IMG_SIZE=64
N_CHANNELS=1

In [ ]:
def add_regularizers(model,
                     kernel_regularization_factor=1e-4,
                     bias_regularization_factor=1e-4,
                     gamma_regularization_factor=1e-4,
                     beta_regularization_factor=1e-4):
    update_model_required = False
    for layer in model.layers:
        if not np.isclose(kernel_regularization_factor, 0.0) and hasattr(
                layer, "kernel_regularizer"):
            update_model_required = True
            layer.kernel_regularizer = regularizers.l2(kernel_regularization_factor)
        if not np.isclose(bias_regularization_factor, 0.0) and hasattr(
                layer, "bias_regularizer"):
            if layer.use_bias:
                update_model_required = True
                layer.bias_regularizer = regularizers.l2(bias_regularization_factor)
        if not np.isclose(gamma_regularization_factor, 0.0) and hasattr(
                layer, "gamma_regularizer"):
            if layer.scale:
                update_model_required = True
                layer.gamma_regularizer = regularizers.l2(gamma_regularization_factor)
        if not np.isclose(beta_regularization_factor, 0.0) and hasattr(
                layer, "beta_regularizer"):
            if layer.center:
                update_model_required = True
                layer.beta_regularizer = regularizers.l2(beta_regularization_factor)
    if update_model_required:
        print("Adding regularizers ...")
        # https://github.com/keras-team/keras/issues/2717#issuecomment-447570737
        vanilla_weights = model.get_weights()
        model = models.model_from_json(model.to_json())
        model.set_weights(vanilla_weights)
    return model

Resnet Model

In [ ]:
# def conv2d_bn(x, nb_filter, kernel_size, strides=(1, 1), padding='same'):
#     x = Conv2D(nb_filter, kernel_size=kernel_size, strides=strides, padding=padding)(x)
# #     x = Conv2D(nb_filter, kernel_size=kernel_size, strides=strides, padding=padding)(x)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)

#     return x
 

# def shortcut(input, residual):
 
#     input_shape = K.int_shape(input)
#     residual_shape = K.int_shape(residual)
#     stride_height = int(round(input_shape[1] / residual_shape[1]))
#     stride_width = int(round(input_shape[2] / residual_shape[2]))
#     equal_channels = input_shape[3] == residual_shape[3]
 
#     identity = input
#     if stride_width > 1 or stride_height > 1 or not equal_channels:
#         identity = Conv2D(filters=residual_shape[3],
#                            kernel_size=(1, 1),
#                            strides=(stride_width, stride_height),
#                            padding="valid",
#                            kernel_regularizer=regularizers.l2(0.0001))(input)
 
#     return add([identity, residual])

In [ ]:
# def basic_block(nb_filter, strides=(1, 1)):
#     def f(input):
 
#         conv1 = conv2d_bn(input, nb_filter, kernel_size=(3, 3), strides=strides)
#         residual = conv2d_bn(conv1, nb_filter, kernel_size=(3, 3))

#         return shortcut(input, residual)
 
#     return f
 
    
# def residual_block(nb_filter, repetitions, is_first_layer=False):
#     def f(input):
#         for i in range(repetitions):
#             strides = (1, 1)
#             if i == 0 and not is_first_layer:
#                 strides = (2, 2)
#             input = basic_block(nb_filter, strides)(input)
#         return input
 
#     return f

In [ ]:
# def resnet_18(input_shape=(64,64,1), nclass=1024):

#     input_ = Input(shape=input_shape)
 
#     conv1 = conv2d_bn(input_, 16, kernel_size=(5, 5), strides=(2, 2))
# #     conv1 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv1)
#     conv1 = BatchNormalization()(conv1)
#     conv1 = Activation('relu')(conv1)
# #     conv1 = Dropout(rate=0.3)(conv1)
    
#     conv1 = residual_block(32, 2, is_first_layer=True)(conv1)
#     conv1 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv1)
#     conv1 = residual_block(64, 2, is_first_layer=True)(conv1)
#     conv1 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv1)
#     conv1 = residual_block(96, 2, is_first_layer=True)(conv1)
#     conv1 = residual_block(128, 2, is_first_layer=True)(conv1)
#     conv1 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv1)
#     conv1 = residual_block(160, 2, is_first_layer=True)(conv1)
#     conv1 = residual_block(192, 2, is_first_layer=True)(conv1)
#     conv1 = residual_block(224, 2, is_first_layer=True)(conv1)
#     conv1 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv1)

# #     conv6 = residual_block(512, 2, is_first_layer=True)(conv5)
 
#     pool2 = GlobalAvgPool2D()(conv1)
# #     pool2 = Dense(nclass, activation='softmax')(pool2)
# #     dense = GlobalAveragePooling2D()(pool2)
# #     dense = Flatten()(dense)
#     dense = Dense(1024, activation = "relu")(pool2)
#     dense = Dropout(rate=0.3)(dense)
#     dense = Dense(512, activation = "relu")(dense)
    
#     head_root = Dense(256, activation = "relu")(dense)
#     head_root = Dense(168, activation = 'softmax')(head_root)
    
#     head_vowel = Dense(128, activation = "relu")(dense)
#     head_vowel = Dense(11, activation = 'softmax')(head_vowel)
    
#     head_consonant = Dense(7, activation = 'softmax')(dense)
    
#     model = Model(inputs=input_, outputs=[head_root, head_vowel, head_consonant])
#     model.summary()
#     model = add_regularizers(model)

#     return model

In [ ]:
# def resblock(inputs,filter_num):
#     out = keras.layers.Conv2D(filter_num, (3, 3), padding='SAME')(inputs)
#     out = keras.layers.Conv2D(filter_num, (3, 3), padding='SAME')(out)
#     out = keras.layers.Conv2D(filter_num, (3, 3), padding='SAME')(out)
#     out = keras.layers.Conv2D(filter_num, (3, 3), padding='SAME')(out)
#     out = keras.layers.BatchNormalization(momentum=0.15)(out)
#     out = keras.layers.Activation('relu')(out)
#     inputs = keras.layers.MaxPool2D(pool_size=(2, 2))(inputs)
#     out = keras.layers.MaxPool2D(pool_size=(2, 2))(out)
    
#     out = keras.layers.Conv2D(filter_num,(5, 5),strides=1,padding='SAME')(out)
#     out = keras.layers.BatchNormalization(momentum=0.15)(out)
#     out = keras.layers.Activation('relu')(out)
#     out = keras.layers.Dropout(rate=0.3)(out)
    
#     identity = keras.layers.Dense(filter_num)(inputs)
#     output = keras.layers.add([out,identity])
#     output = keras.layers.Activation('relu')(output)
#     return output

# def my_resnet(inputs):
#     model = resblock(inputs,32)
#     model = resblock(model,64)
#     model = resblock(model,128)
#     model = resblock(model,256)
    
#     identity = keras.layers.Conv2D(256, (3, 3), padding='SAME', activation='relu', input_shape=(2, 2, 1))(inputs)
#     identity = keras.layers.BatchNormalization(momentum=0.15)(identity)
#     identity = keras.layers.Activation('relu')(identity)
#     identity = keras.layers.Dropout(rate=0.3)(identity)
#     output = keras.layers.add([model,identity])
#     output = keras.layers.Activation('relu')(output)
#     return output
# #     return model

In [ ]:
# # preprocessing
# inputs = keras.layers.Input(shape = (IMG_SIZE, IMG_SIZE, 1))
# model = keras.layers.Conv2D(16, (3, 3), padding='SAME', activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1))(inputs)
# model = keras.layers.Conv2D(16, (3, 3), padding='SAME', activation='relu')(model)
# model = keras.layers.Conv2D(16, (3, 3), padding='SAME', activation='relu')(model)
# model = keras.layers.BatchNormalization(momentum=0.15)(model)
# model = keras.layers.MaxPool2D(pool_size=(2, 2))(model)
# model = keras.layers.Conv2D(filters=32, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
# model = keras.layers.Dropout(rate=0.3)(model)

# # iterate the resblock
# model = my_resnet(model)

# # only for root
# # dense0 = resblock(model,320)
# # dense0 = keras.layers.GlobalAveragePooling2D()(dense0)
# # dense0 = keras.layers.Flatten()(dense0)
# # dense0 = keras.layers.Dense(1024, activation = "relu")(dense0)
# # dense0 = keras.layers.Dropout(rate=0.3)(dense0)
# # dense0 = keras.layers.Dense(512, activation = "relu")(dense0)
# # head_root = keras.layers.Dense(168, activation = 'softmax')(dense0)

# # adaptive other two
# dense1 = keras.layers.GlobalAveragePooling2D()(model)
# dense1 = keras.layers.Flatten()(dense1)
# dense1 = keras.layers.Dense(1024, activation = "relu")(dense1)
# dense1 = keras.layers.Dropout(rate=0.3)(dense1)
# dense1 = keras.layers.Dense(512, activation = "relu")(dense1)
# head_root = keras.layers.Dense(168, activation = 'softmax')(dense1)
# head_vowel = keras.layers.Dense(11, activation = 'softmax')(dense1)
# head_consonant = keras.layers.Dense(7, activation = 'softmax')(dense1)

# model = keras.models.Model(inputs=inputs, outputs=[head_root, head_vowel, head_consonant])

# model.summary()
# model = add_regularizers(model)
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model = resnet_18()
# adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
# model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# plot_model(model, to_file='model.png')

In [ ]:
def resize(df, size=64, need_progress_bar=True):
    resized = {}
    resize_size=64
    if need_progress_bar:
        l = tqdm(range(df.shape[0]))
    else:
        l = range(df.shape[0])
        
    for i in l:
        image=df.loc[df.index[i]].values.reshape(137,236)
        _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]

        idx = 0 
        ls_xmin = []
        ls_ymin = []
        ls_xmax = []
        ls_ymax = []
        for cnt in contours:
            idx += 1
            x,y,w,h = cv2.boundingRect(cnt)
            ls_xmin.append(x)
            ls_ymin.append(y)
            ls_xmax.append(x + w)
            ls_ymax.append(y + h)
        xmin = min(ls_xmin)
        ymin = min(ls_ymin)
        xmax = max(ls_xmax)
        ymax = max(ls_ymax)

        roi = image[ymin:ymax,xmin:xmax]
        resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
        resized[df.index[i]] = resized_roi.reshape(-1)
    
    del l
    resized = pd.DataFrame(resized).T
#     return cv2.normalize(resized, None, 0, 1, cv2.NORM_MINMAX)
    return resized


def get_dummies(df):
    cols = []
    for col in df:
        cols.append(pd.get_dummies(df[col].astype(str)))
    return pd.concat(cols, axis=1)

In [ ]:
batch_size = 256
epochs = 2

In [ ]:
class MultiOutputDataGenerator(ImageDataGenerator):

    def flow(self,
             x,
             y=None,
             batch_size=32,
             shuffle=True,
             sample_weight=None,
             seed=None,
             save_to_dir=None,
             save_prefix='',
             save_format='png',
             subset=None):

        targets = None
        target_lengths = {}
        ordered_outputs = []
        for output, target in y.items():
            if targets is None:
                targets = target
            else:
                targets = np.concatenate((targets, target), axis=1)
            target_lengths[output] = target.shape[1]
            ordered_outputs.append(output)


        for flowx, flowy in super().flow(x, targets, batch_size=batch_size,
                                         shuffle=shuffle):
            target_dict = {}
            i = 0
            for output in ordered_outputs:
                target_length = target_lengths[output]
                target_dict[output] = flowy[:, i: i + target_length]
                i += target_length

            yield flowx, target_dict

In [ ]:
HEIGHT = 137
WIDTH = 236

In [ ]:
# os.listdir('/kaggle/input')

In [ ]:
factors = [0.75,0.5,0.45,0.4,0.35,0.3,0.25,0.2,0.1]
# Set a learning rate annealer. Learning rate will be half after 3 epochs if accuracy is not increased
learning_rate_reduction_root = ReduceLROnPlateau(monitor='dense_3_accuracy', 
                                            factor=factors[0],
                                            patience=1,
                                            cooldown=1,
                                            verbose=1,
                                            min_lr=0.000001)
# learning_rate_reduction_vowel = ReduceLROnPlateau(monitor='dense_6_accuracy', 
#                                             factor=factors[1],
#                                             patience=2, 
#                                             cooldown=1,
#                                             verbose=1,
#                                             min_lr=0.00001)
# learning_rate_reduction_consonant = ReduceLROnPlateau(monitor='dense_7_accuracy', 
#                                             factor=factors[0],
#                                             patience=1, 
#                                             cooldown=1,
#                                             verbose=1,
#                                             min_lr=0.00005)

In [ ]:
# # FROM HERE!!!!!!!!!!!
# df_a = pd.read_parquet(f'/kaggle/input/bengaliai-cv19/train_image_data_0.parquet')
# df_c = pd.read_parquet(f'/kaggle/input/bengaliai-cv19/train_image_data_2.parquet')
# df_ac = pd.concat([df_a,df_c])
# del df_a, df_c

# train_df = pd.merge(df_ac, train_df_, on='image_id').drop(['image_id'], axis=1)
# del df_ac
# gc.collect()

In [ ]:
# X_train = train_df.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic'], axis=1)
# Y_train_root = pd.get_dummies(train_df['grapheme_root']).values
# Y_train_vowel = pd.get_dummies(train_df['vowel_diacritic']).values
# Y_train_consonant = pd.get_dummies(train_df['consonant_diacritic']).values
# del train_df
# gc.collect()
# X_train = resize(X_train).astype(np.float32)/255

In [ ]:
# get the model
model = models.load_model('/kaggle/input/10thmodel/my_model10_12.h5')
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.000002)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# # CNN takes images in shape `(batch_size, h, w, channels)`, so reshape the images
# X_train = X_train.values.reshape(-1, IMG_SIZE, IMG_SIZE, N_CHANNELS)
# gc.collect()

# print(f'Training images: {X_train.shape}')
# print(f'Training labels root: {Y_train_root.shape}')
# print(f'Training labels vowel: {Y_train_vowel.shape}')
# print(f'Training labels consonants: {Y_train_consonant.shape}')

# # Divide the data into training and validation set
# x_train, x_test, y_train_root, y_test_root, y_train_vowel, y_test_vowel, y_train_consonant, y_test_consonant = train_test_split(X_train, Y_train_root, Y_train_vowel, Y_train_consonant, test_size=0.08, random_state=666)
# # del X_train
# # del Y_train_root, Y_train_vowel, Y_train_consonant

# # Data augmentation for creating more training data
# datagen = MultiOutputDataGenerator(
#     featurewise_center=False,  # set input mean to 0 over the dataset
#     samplewise_center=False,  # set each sample mean to 0
#     featurewise_std_normalization=False,  # divide inputs by std of the dataset
#     samplewise_std_normalization=False,  # divide each input by its std
#     zca_whitening=False,  # apply ZCA whitening
#     rotation_range=8,  # randomly rotate images in the range (degrees, 0 to 180)
#     zoom_range = 0.15, # Randomly zoom image 
#     width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
#     height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
#     horizontal_flip=False,  # randomly flip images
#     vertical_flip=False)  # randomly flip images


# # This will just calculate parameters required to augment the given data. This won't perform any augmentations
# datagen.fit(x_train)

# # Fit the model
# history = model.fit_generator(datagen.flow(x_train, {'dense_3': y_train_root, 'dense_4': y_train_vowel, 'dense_5': y_train_consonant}, batch_size=batch_size),
#                           epochs = epochs, validation_data = (x_test, [y_test_root, y_test_vowel, y_test_consonant]), 
#                           steps_per_epoch=x_train.shape[0] // batch_size, 
#                           callbacks=[learning_rate_reduction_root])

# # Delete to reduce memory usage
# del x_train
# del x_test
# del y_train_root
# del y_test_root
# del y_train_vowel
# del y_test_vowel
# del y_train_consonant
# del y_test_consonant
# del datagen

# gc.collect()

In [ ]:
# os.makedirs('./save_weights')
# # history = load_parquet(train_df, IMG_SIZE, N_CHANNELS, 0)
# model.save('./save_weights/my_model10_9.h5')
# histories = []
# histories.append(history)

In [ ]:
# def load_parquet(train_df_, IMG_SIZE, N_CHANNELS, i):
#     train_df = pd.merge(pd.read_parquet(f'/kaggle/input/bengaliai-cv19/train_image_data_{i}.parquet'), train_df_, on='image_id').drop(['image_id'], axis=1)
#     X_train = train_df.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic'], axis=1)
#     X_train = resize(X_train).astype(np.float64)/255
# #     X_train = cv2.normalize(X_train, None, 0, 1, cv2.NORM_MINMAX)
# #     X_train = resize(X_train)
#     gc.collect()
    
#     # CNN takes images in shape `(batch_size, h, w, channels)`, so reshape the images
#     X_train = X_train.values.reshape(-1, IMG_SIZE, IMG_SIZE, N_CHANNELS)
    
#     Y_train_root = pd.get_dummies(train_df['grapheme_root']).values
#     Y_train_vowel = pd.get_dummies(train_df['vowel_diacritic']).values
#     Y_train_consonant = pd.get_dummies(train_df['consonant_diacritic']).values
#     del train_df

#     print(f'Training images: {X_train.shape}')
#     print(f'Training labels root: {Y_train_root.shape}')
#     print(f'Training labels vowel: {Y_train_vowel.shape}')
#     print(f'Training labels consonants: {Y_train_consonant.shape}')

#     # Divide the data into training and validation set
#     x_train, x_test, y_train_root, y_test_root, y_train_vowel, y_test_vowel, y_train_consonant, y_test_consonant = train_test_split(X_train, Y_train_root, Y_train_vowel, Y_train_consonant, test_size=0.08, random_state=666)
#     del X_train
#     del Y_train_root, Y_train_vowel, Y_train_consonant

#     # Data augmentation for creating more training data
#     datagen = MultiOutputDataGenerator(
#         featurewise_center=False,  # set input mean to 0 over the dataset
#         samplewise_center=False,  # set each sample mean to 0
#         featurewise_std_normalization=False,  # divide inputs by std of the dataset
#         samplewise_std_normalization=False,  # divide each input by its std
#         zca_whitening=False,  # apply ZCA whitening
#         rotation_range=8,  # randomly rotate images in the range (degrees, 0 to 180)
#         zoom_range = 0.15, # Randomly zoom image 
#         width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
#         height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
#         horizontal_flip=False,  # randomly flip images
#         vertical_flip=False)  # randomly flip images


#     # This will just calculate parameters required to augment the given data. This won't perform any augmentations
#     datagen.fit(x_train)

#     # Fit the model
#     history = model.fit_generator(datagen.flow(x_train, {'dense_4': y_train_root, 'dense_6': y_train_vowel, 'dense_7': y_train_consonant}, batch_size=batch_size),
#                               epochs = epochs, validation_data = (x_test, [y_test_root, y_test_vowel, y_test_consonant]), 
#                               steps_per_epoch=x_train.shape[0] // batch_size, 
#                               callbacks=[learning_rate_reduction_root])
    
#     # Delete to reduce memory usage
#     del x_train
#     del x_test
#     del y_train_root
#     del y_test_root
#     del y_train_vowel
#     del y_test_vowel
#     del y_train_consonant
#     del y_test_consonant
#     del datagen
    
#     gc.collect()
#     return history

In [ ]:
# os.makedirs('./save_weights')
# history = load_parquet(train_df_, IMG_SIZE, N_CHANNELS, 0)
# model.save('./save_weights/my_model9_0.h5')
# histories = []
# histories.append(history)
# history = load_parquet(train_df_, IMG_SIZE, N_CHANNELS, 1)
# model.save('./save_weights/my_model8_9.h5')
# histories.append(history)
# # # history = load_parquet(train_df_, IMG_SIZE, N_CHANNELS, 2)
# # # model.save('./save_weights/my_model1_2.h5')
# # # history = load_parquet(train_df_, IMG_SIZE, N_CHANNELS, 3)
# # # model.save('./save_weights/my_model1_3.h5')

In [ ]:
# adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
# model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# history = load_parquet(train_df_, IMG_SIZE, N_CHANNELS, 1)
# model.save('./save_weights/my_model9_1.h5')
# histories.append(history)

In [ ]:
# histories = []
# model0 = keras.models.load_model('/kaggle/input/pretrained-model/my_model1_0.h5')
# model1 = keras.models.load_model('/kaggle/input/pretrained-model/my_model1_1.h5')
# model2 = keras.models.load_model('/kaggle/input/pretrained-model/my_model1_2.h5')
# model3 = keras.models.load_model('/kaggle/input/pretrained-model/my_model1_3.h5')
# histories.append(model0)
# histories.append(model1)
# histories.append(model2)
# histories.append(model3)
# del model0
# del model1
# del model2

In [ ]:
%matplotlib inline
def plot_loss(his, epoch, title):
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(np.arange(0, epoch), his.history['loss'], label='train_loss')
    plt.plot(np.arange(0, epoch), his.history['dense_4_loss'], label='train_root_loss')
    plt.plot(np.arange(0, epoch), his.history['dense_6_loss'], label='train_vowel_loss')
    plt.plot(np.arange(0, epoch), his.history['dense_7_loss'], label='train_consonant_loss')
    
    plt.plot(np.arange(0, epoch), his.history['val_dense_4_loss'], label='val_train_root_loss')
    plt.plot(np.arange(0, epoch), his.history['val_dense_6_loss'], label='val_train_vowel_loss')
    plt.plot(np.arange(0, epoch), his.history['val_dense_7_loss'], label='val_train_consonant_loss')
    
    plt.title(title)
    plt.xlabel('Epoch #')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.show()

def plot_acc(his, epoch, title):
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(np.arange(0, epoch), his.history['dense_4_accuracy'], label='train_root_acc')
    plt.plot(np.arange(0, epoch), his.history['dense_6_accuracy'], label='train_vowel_accuracy')
    plt.plot(np.arange(0, epoch), his.history['dense_7_accuracy'], label='train_consonant_accuracy')
    
    plt.plot(np.arange(0, epoch), his.history['val_dense_4_accuracy'], label='val_root_acc')
    plt.plot(np.arange(0, epoch), his.history['val_dense_6_accuracy'], label='val_vowel_accuracy')
    plt.plot(np.arange(0, epoch), his.history['val_dense_7_accuracy'], label='val_consonant_accuracy')
    plt.title(title)
    plt.xlabel('Epoch #')
    plt.ylabel('Accuracy')
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
# for dataset in range(1):
#     plot_loss(histories[dataset], epochs, f'Training Dataset: {dataset}')
#     plot_acc(histories[dataset], epochs, f'Training Dataset: {dataset}')
# # plot_loss(histories[0], epochs, f'Training Dataset: {0}')
# # plot_acc(histories[0], epochs, f'Training Dataset: {0}')
# del histories
# gc.collect()

In [ ]:
preds_dict = {
    'grapheme_root': [],
    'vowel_diacritic': [],
    'consonant_diacritic': []
}

In [ ]:
# model = keras.models.load_model('/kaggle/input/pretrained-model/my_model1_3.h5')
components = ['consonant_diacritic', 'grapheme_root', 'vowel_diacritic']
target=[] # model predictions placeholder
row_id=[] # row_id place holder
for i in range(4):
    df_test_img = pd.read_parquet('/kaggle/input/bengaliai-cv19/test_image_data_{}.parquet'.format(i)) 
    df_test_img.set_index('image_id', inplace=True)

    X_test = resize(df_test_img, need_progress_bar=False)/255
    X_test = X_test.values.reshape(-1, IMG_SIZE, IMG_SIZE, N_CHANNELS)
    
    preds = model.predict(X_test)

    for i, p in enumerate(preds_dict):
        preds_dict[p] = np.argmax(preds[i], axis=1)

    for k,id in enumerate(df_test_img.index.values):  
        for i,comp in enumerate(components):
            id_sample=id+'_'+comp
            row_id.append(id_sample)
            target.append(preds_dict[comp][k])
    del df_test_img
    del X_test
    gc.collect()

df_sample = pd.DataFrame(
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)
df_sample.to_csv('submission.csv',index=False)
df_sample.head()